## Matrix Multiplication using global memory

In [2]:


from __future__ import division
from numba import cuda, float32
from timeit import default_timer as time

import numpy
import math

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 32

@cuda.jit
def fmatmul(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    i, j = cuda.grid(2)
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp
        

A = numpy.full((6400, 9600), 0.3, numpy.float)
B = numpy.full((9600, 3200), 0.4, numpy.float)

A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.device_array((A.shape[0], B.shape[1]))

print ('After allocating memory')
print cuda.current_context().get_memory_info()

# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)


# Start the kernel 

s = time()
fmatmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
e = time()

res = C_global_mem.copy_to_host()


cuda.current_context().deallocations.clear()

tcuda = e - s
print ('deallocating memory')
print cuda.current_context().get_memory_info()


# Host compute
Amat = numpy.matrix(A)
Bmat = numpy.matrix(B)

s = time()
Cans = numpy.dot(Amat,Bmat)
e = time()
tcpu = e - s

print('cpu:  %f' % tcpu)
print('cuda: %f' % tcuda)
print('cuda speedup: %.2fx' % (tcpu / tcuda))
# Check result
assert numpy.allclose(res, Cans)

After allocating memory
_MemoryInfo(free=5320920268L, total=8589934592L)
deallocating memory
_MemoryInfo(free=6219943116L, total=8589934592L)
cpu:  4.368995
cuda: 0.164125
cuda speedup: 26.62x



# Below is the mat mul with shared memory

In [3]:
from __future__ import division
from numba import cuda, float32
import numpy
import math
from timeit import default_timer as time
# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
print ('initial memory')
print cuda.current_context().get_memory_info()


# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 32

        
@cuda.jit
def fast_matmul(A, B, C):
    """
    Perform matrix multiplication of C = A * B
    Each thread computes one element of the result matrix C
    """

    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(int(A.shape[1] / TPB)):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()



    C[x, y] = tmp

        
# The data array
A = numpy.full((6400, 9600), 0.3, numpy.float)
B = numpy.full((9600, 3200), 0.4, numpy.float)
"""
e=time()
if(B.shape[0]%32!=0):
    temp1=B.shape[0]
    B=numpy.append(B,numpy.zeros((max(B.shape[0]%32,(32-B.shape[0])%32),B.shape[1])),axis=0)
if(A.shape[1]%32!=0):
    temp2=A.shape[1]
    A=numpy.append(A,numpy.zeros((A.shape[0],max(A.shape[1]%32,(32-A.shape[1])%32))),axis=1)
t=time()
"""


A_global_mem = cuda.to_device(A)
B_global_mem = cuda.to_device(B)
C_global_mem = cuda.device_array((A.shape[0], B.shape[1]))
print ('After allocating memory')
print cuda.current_context().get_memory_info()

# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(B.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)


# Start the kernel 

s = time()
fast_matmul[blockspergrid, threadsperblock](A_global_mem, B_global_mem, C_global_mem)
e = time()

res = C_global_mem.copy_to_host()
#res=numpy.delete(res,range(temp2,B.shape[1]),axis=1)
#res = C_global_mem.copy_to_host()

cuda.current_context().deallocations.clear()

tcuda = e - s
print ('deallocating memory')
print cuda.current_context().get_memory_info()


# Host compute
Amat = numpy.matrix(A)
Bmat = numpy.matrix(B)

s = time()
Cans = numpy.dot(Amat,Bmat)
e = time()
tcpu = e - s
#Cans=numpy.delete(Cans,range(temp2,B.shape[1]),axis=1)

print('cpu:  %f' % tcpu)
print('cuda: %f' % tcuda)
print('cuda speedup: %.2fx' % (tcpu / tcuda))
# Check result
assert numpy.allclose(res, Cans)


initial memory
_MemoryInfo(free=6219943116L, total=8589934592L)
After allocating memory
_MemoryInfo(free=5318823116L, total=8589934592L)
deallocating memory
_MemoryInfo(free=6219943116L, total=8589934592L)
cpu:  4.209886
cuda: 0.291502
cuda speedup: 14.44x


# Below is the mat transpose with shared memory

In [5]:
from numba import cuda, float32
import numpy
import math
from timeit import default_timer as time

TPB =32
TILE_DIM = 16
BLOCK_ROWS = 8

import numba.types

TILE_DIM_PADDED = TILE_DIM + 1 

@cuda.jit
def tile_transpose(a_in, a_out):
    # THIS CODE ASSUMES IT IS RUNNING WITH A BLOCK DIMENSION OF (TILE_SIZE x TILE_SIZE)
    # AND INPUT IS A MULTIPLE OF TILE_SIZE DIMENSIONS
    tile = cuda.shared.array((TILE_DIM, TILE_DIM_PADDED), numba.types.int32)

    x = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.y
    
    for j in range(0, TILE_DIM, BLOCK_ROWS):
        tile[cuda.threadIdx.y + j, cuda.threadIdx.x] = a_in[y + j, x] # transpose tile into shared memory

    cuda.syncthreads()  # wait for all threads in the block to finish updating shared memory

    #Compute transposed offsets
    x = cuda.blockIdx.y * TILE_DIM + cuda.threadIdx.x
    y = cuda.blockIdx.x * TILE_DIM + cuda.threadIdx.y

    for j in range(0, TILE_DIM, BLOCK_ROWS):
        a_out[y + j, x] = tile[cuda.threadIdx.x, cuda.threadIdx.y + j];

@cuda.jit
def fast_transpose(A,output):
    
    #tile = cuda.shared.array(shape=tile_shape, dtype=dt)
    tile = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x * cuda.blockDim.x
    by = cuda.blockIdx.y * cuda.blockDim.y
    x = by + tx
    y = bx + ty

    if by+ty < A.shape[0] and bx+tx < A.shape[1]:
        tile[ty, tx] = A[by+ty, bx+tx]
    cuda.syncthreads()
    if y < output.shape[0] and x < output.shape[1]:
        output[y, x] = tile[tx, ty]
size=1024        
        
A = numpy.arange(size*size, dtype=numpy.int32).reshape((size, size)) # [32 x 48] matrix containing all 3's


A_global = cuda.to_device(A)
C_global=  cuda.device_array_like(A) # [32 x 16] matrix result
print ('After allocating memory')
print cuda.current_context().get_memory_info()

# Configure the blocks
threadsperblock = (TPB, TPB)
blockspergrid_x = int(math.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(math.ceil(A.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

grid_shape = (int(size/TILE_DIM), int(size/TILE_DIM))

# Start the kernel 

s = time()
tile_transpose[grid_shape,(TILE_DIM, BLOCK_ROWS)](A_global, C_global)
e=time()
cuda.synchronize()

res = C_global.copy_to_host()
#res=numpy.delete(res,range(temp2,B.shape[1]),axis=1)
#res = C_global_mem.copy_to_host()

cuda.current_context().deallocations.clear()

tcuda = e - s
print ('deallocating memory')
print cuda.current_context().get_memory_info()


# Host compute
#Amat = numpy.matrix(A)

s = time()
Cans = numpy.transpose(A)
e = time()
tcpu = e - s
#Cans=numpy.delete(Cans,range(temp2,B.shape[1]),axis=1)

print('cpu:  %f' % tcpu)
print('cuda: %f' % tcuda)
print('cuda speedup: %.2fx' % (tcpu / tcuda))
# Check result
assert numpy.allclose(res, Cans)
        


After allocating memory
_MemoryInfo(free=6203165900L, total=8589934592L)
deallocating memory
_MemoryInfo(free=6211554508L, total=8589934592L)
cpu:  0.001202
cuda: 0.216642
cuda speedup: 0.01x


### Speed-up trial for numpy.sum()

In [ ]:
from numba import cuda, float32
from timeit import default_timer as time

import numpy
import math

TPB =32

@cuda.jit
def fast_add(A, B, C):
     x, y = cuda.grid(2)
     if x < C.shape[0] and y < C.shape[1]:
         C[x, y] = A[x, y] + B[x, y]
            
A = numpy.full((7000, 1), 3, numpy.float) # [32 x 48] matrix containing all 3's
B = numpy.full((7000, 1), 4, numpy.float)

threadsperblock = (TPB,TPB)
blockspergrid_x = int(math.ceil(A.shape[0])/ threadsperblock[1])
blockspergrid_y = int(math.ceil(A.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)

d_arr1 = cuda.to_device(A)
d_arr2 = cuda.to_device(B)
d_result = cuda.device_array_like(d_arr1)

s = time()
fast_add[blockspergrid, threadsperblock](d_arr1,d_arr2,d_result)
e = time()

res = d_result.copy_to_host()

tgpu=e-s

s = time()
Cans = numpy.sum((A,B),axis=1)
e = time()
tcpu = e - s
print("Cpu"+str(tcpu))
print("Gpu"+str(tgpu))
print("speedup"+str(tcpu/tgpu))
